# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '..' not in sys.path:
    sys.path.insert(0, '..')

In [3]:
from src.m_utils import constants as cts
from src.base.data_loaders.data_loader import DLName
from src.base.gt_loaders.gt_names import GTName
from src.exp_runner import ExperimentRunner

from src.base.experiment.dataset.benchmark_dataset import BenchmarkDataset
from src.base.experiment.evaluation.model_evaluator import DataSource, DataPredSelection
from src.base.experiment.training.base_models import BaseModel
from src.base.experiment.training.optimizers import Optimizer

from src.m_utils.mtl_approach import MTLApproach
from src.m_utils.nas_mtl_approach import NAS_MTLApproach
from src.base.experiment.tasks.task import ICAO_REQ, MNIST_TASK

 ==> Restrict GPU memory growth: True


# Start Network runner

In [4]:
kwargs = { 
    'use_neptune': True,
    'exp_params' : {
        'name': 'neural_arch_search',
        'description': 'NAS Approach 2 with FVC dataset with 50 trials and patience and 1 child epoch',
        'tags': ['nas', 'nas_approach_1', 'benchamark', 'mnist', '1_child_epochs'],
        'src_files': ["../src/**/*.py"]
    },
    'properties': {
        'approach': NAS_MTLApproach.APPROACH_2,
        'benchmarking': {
            'use_benchmark_data': False,
            'benchmark_dataset': BenchmarkDataset.MNIST,
            'tasks': list(MNIST_TASK)
        },
        'icao_data': {
            'icao_gt': {
                'use_gt_data': True,
                'gt_names': {
                    'train_validation': [],
                    'test': [],
                    'train_validation_test': [GTName.FVC]
                },
            },
            'icao_dl': {
                'use_dl_data': False,
                'tagger_model': None
            },
            'reqs': list(ICAO_REQ),
            'aligned': False
        },
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'exec_nas': True,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'net_train_params': {
        'base_model': BaseModel.MOBILENET_V2,
        'batch_size': 32,
        'n_epochs': 1,
        'early_stopping': 5,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'dropout': 0.3
    },
    'nas_params': {
        'max_blocks_per_branch': 5,
        'n_child_epochs': 1,
        'controller_epochs': 1,
        'controller_batch_size': 32,
        'n_trials': 50
    }
}

runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 90559
Process ID: 102485
---------------------------
-----
Use Neptune:  True
-----
-------------------
Args: 
{'exp_params': {'description': 'NAS Approach 2 with FVC dataset with 50 trials '
                               'and patience and 1 child epoch',
                'name': 'neural_arch_search',
                'src_files': ['../src/**/*.py'],
                'tags': ['nas',
                         'nas_approach_1',
                         'benchamark',
                         'mnist',
                         '1_child_epochs']},
 'nas_params': {'controller_batch_size': 32,
                'controller_epochs': 1,
                'max_blocks_per_branch': 5,
                'n_child_epochs': 1,
                'n_trials': 50},
 'net_train_params': {'base_model': <BaseModel.MOBILENET_V2: {'name': 'mobilnet_v2', 'target_size': (224, 224), 'prep_function': <function preproc

/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


# Load Data

In [5]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4928, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


In [6]:
runner.train_data.head()

,origin,img_name,mouth,rotation,l_away,eyes_closed,close,hat,dark_glasses,frames_heavy,...,sh_head,blurred,ink_mark,skin_tone,washed_out,pixelation,hair_eyes,background,red_eyes,aligned
0,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
1,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
2,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
3,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,False
4,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,False


# <font color='red'>Producing Fake Data</font>

# Data Generators

In [7]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 4928 validated image filenames.
Found 547 validated image filenames.
Found 288 validated image filenames.
TOTAL: 5763

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Setup Experiment

In [8]:
runner.setup_experiment()

-------------------- create experiment -------------------
Setup neptune properties and parameters
Properties and parameters setup done!


# Labels Distribution

In [9]:
%%capture
runner.summary_labels_dist()

# Neural Architecture Search

In [ ]:
runner.run_neural_architeture_search()

-------------------- run neural architecture search -------------------
Executing neural architectural search
  Memory reseted

==================== STARTING NEW TRIAL ====================
 selecting new config...
  Memory is empty
 controller_pred: [[0.22257014 0.2246993  0.26717    0.28556052]]


 ------ Training 1 | Config: {'n_denses_0': 2, 'n_denses_1': 2, 'n_denses_2': 2, 'n_denses_3': 2} -----

Creating model...
Model created
Training MOBILENET_V2 network
 .. Not fine tuning base model...
  .. Total params: 4,242,286
  .. Trainable params: 1,984,302
  .. Non-trainable params: 2,257,984

Epoch 00001: val_loss improved from inf to 2.56555, saving model to training_ckpt/best_model.hdf5


/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


..Loading best model
..Checkpoint weights loaded
Testing Trained Model
Predicting labels....
18/18 [==============================] - 33s 2s/step
Prediction finished!


/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:803: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


  Task  0: mouth           | EER_mean: 0.22830 | EER_interp: 0.22840 | ACC: 0.77150
  Task  1: rotation        | EER_mean: 0.34630 | EER_interp: 0.34470 | ACC: 0.65450
  Task  2: l_away          | EER_mean: 0.23560 | EER_interp: 0.23300 | ACC: 0.76600
  Task  3: eyes_closed     | EER_mean: 0.06600 | EER_interp: 0.06480 | ACC: 0.93600
  Task  4: close           | EER_mean: 0.40810 | EER_interp: 0.00000 | ACC: 0.01100
  Task  5: hat             | EER_mean: 0.07530 | EER_interp: 0.07290 | ACC: 0.92870
  Task  6: dark_glasses    | EER_mean: 0.03640 | EER_interp: 0.03740 | ACC: 0.96340
  Task  7: frames_heavy    | EER_mean: 0.50090 | EER_interp: 0.00000 | ACC: 0.00550
  Task  8: frame_eyes      | EER_mean: 0.14680 | EER_interp: 0.14580 | ACC: 0.85370
  Task  9: flash_lenses    | EER_mean: 0.15040 | EER_interp: 0.15240 | ACC: 0.84640
  Task 10: veil            | EER_mean: 0.44160 | EER_interp: 0.00000 | ACC: 0.06030
  Task 11: reflection      | EER_mean: 0.25660 | EER_interp: 0.25600 | ACC: 

...

  Task  0: mouth           | EER_mean: 0.18900 | EER_interp: 0.19100 | ACC: 0.80800
  Task  1: rotation        | EER_mean: 0.37230 | EER_interp: 0.37880 | ACC: 0.61790
  Task  2: l_away          | EER_mean: 0.21210 | EER_interp: 0.21340 | ACC: 0.78610
  Task  3: eyes_closed     | EER_mean: 0.09980 | EER_interp: 0.09240 | ACC: 0.90310
  Task  4: close           | EER_mean: 0.15950 | EER_interp: 0.00000 | ACC: 0.01100
  Task  5: hat             | EER_mean: 0.24800 | EER_interp: 0.00000 | ACC: 0.17000
  Task  6: dark_glasses    | EER_mean: 0.01920 | EER_interp: 0.01360 | ACC: 0.99090
  Task  7: frames_heavy    | EER_mean: 0.50000 | EER_interp: 0.00000 | ACC: 0.00550
  Task  8: frame_eyes      | EER_mean: 0.12500 | EER_interp: 0.12580 | ACC: 0.87390
  Task  9: flash_lenses    | EER_mean: 0.15670 | EER_interp: 0.15360 | ACC: 0.84460
  Task 10: veil            | EER_mean: 0.21430 | EER_interp: 0.00000 | ACC: 0.06030
  Task 11: reflection      | EER_mean: 0.21890 | EER_interp: 0.21940 | ACC: 

/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


..Loading best model
..Checkpoint weights loaded
Testing Trained Model
Predicting labels....
18/18 [==============================] - 33s 2s/step
Prediction finished!


/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:803: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


  Task  0: mouth           | EER_mean: 0.26770 | EER_interp: 0.26840 | ACC: 0.73130
  Task  1: rotation        | EER_mean: 0.35770 | EER_interp: 0.35940 | ACC: 0.63990
  Task  2: l_away          | EER_mean: 0.19900 | EER_interp: 0.19650 | ACC: 0.80260
  Task  3: eyes_closed     | EER_mean: 0.09430 | EER_interp: 0.09140 | ACC: 0.91040
  Task  4: close           | EER_mean: 0.50050 | EER_interp: 0.00000 | ACC: 0.01100
  Task  5: hat             | EER_mean: 0.37250 | EER_interp: 0.00000 | ACC: 0.17000
  Task  6: dark_glasses    | EER_mean: 0.02420 | EER_interp: 0.02170 | ACC: 0.97620
  Task  7: frames_heavy    | EER_mean: 0.50000 | EER_interp: 0.00000 | ACC: 0.00550
  Task  8: frame_eyes      | EER_mean: 0.13820 | EER_interp: 0.14000 | ACC: 0.85920
  Task  9: flash_lenses    | EER_mean: 0.15670 | EER_interp: 0.15800 | ACC: 0.84280
  Task 10: veil            | EER_mean: 0.47810 | EER_interp: 0.00000 | ACC: 0.06030
  Task 11: reflection      | EER_mean: 0.28300 | EER_interp: 0.28340 | ACC: 

# Create Model with Best Config Found

In [ ]:
runner.create_model()

# Visualize Model

In [ ]:
runner.visualize_model(outfile_path=f"figs/nas/nas_model_approach_1.png")

In [ ]:
%%capture
runner.model_summary()

# Training Model

In [ ]:
runner.train_model()

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

## Validation Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model(verbose=False)

## Test Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model(verbose=False)

# Visualize Model Classification

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()

# Testing Network Modification

# Test - Customized Loss Function

# Test - 